## TensorFlow キャッチアップ

In [0]:
import tensorflow as tf
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

In [21]:
# constant

t1 = time.time() 

# 計算開始

a = tf.constant(5)
b = tf.constant(7)
add = tf.add(a, b)

print(add)                               # エッジの説明が格納されているだけ

with tf.Session() as sess:
    output = sess.run(add)     # Session.run()にエッジを格納すると計算がなされる
    print(output) # 12
sess.close()

# 計算終了


t2 = time.time()
elapsed_time = t2-t1
print(f"経過時間：{elapsed_time}")

Tensor("Add_14:0", shape=(), dtype=int32)
12
経過時間：0.012064456939697266


In [17]:
# 処理前の時刻
t1 = time.time() 

# 計算開始

a_n = np.array(5)
b_n = np.array(7)
output_n = np.add(a_n, b_n)
print(output_n) # 12

# 計算終了


t2 = time.time()
elapsed_time = t2-t1
print(f"経過時間：{elapsed_time}")

# 簡単な計算だとNumpyのが早い・記述も楽

12
経過時間：0.0006110668182373047


In [28]:
# placeholder

t1 = time.time() 

# 計算開始

c = tf.placeholder(tf.int32)
d = tf.placeholder(tf.int32)
add = tf.add(c, d)

with tf.Session() as sess:
    output = sess.run(add, feed_dict={c:5, d:7})
    print(output) # 12
sess.close()
# 計算終了

t2 = time.time()
elapsed_time = t2-t1
print(f"経過時間：{elapsed_time}")

12
経過時間：0.01483917236328125


- ロジスティック回帰

In [0]:
# 入力が1, 1の場合にのみ1と出力したい
x_train = np.array([[0,0],[0,1],[1,0],[1,1]])
y_train = np.array([[0],[0],[0],[1]])

x = tf.placeholder(tf.float32, [None, 2])
t = tf.placeholder(tf.float32, [None, 1])

In [0]:
# Valiable → 学習により更新を行う値であることを宣言(重み・バイアスなど)

W = tf.Variable(tf.zeros([2,1]))
b = tf.Variable(tf.zeros([1]))

In [0]:
# matmul → np.dotに近い挙動
y = tf.sigmoid(tf.matmul(x, W) + b)

# クロスエントロピー誤差
cross_entropy = tf.reduce_sum(-t * tf.log(y) - (1 - t) * tf.log(1 - y))

# 二乗和誤差
sse = tf.reduce_sum(tf.square(y - t))

# 勾配降下法(0.1は学習率)
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

# 予測・正確度
correct_prediction = tf.equal(tf.sign(y - 0.5), tf.sign(t - 0.5))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# 上記のエッジはrunできんかった

#with tf.Session() as sess:
    #output1 = sess.run(y)
    #output2 = sess.run(cross_entropy)
    #output3 = sess.run(sse)
    #output4 = sess.run(train_step)
    #output5 = sess.run(correct_prediction)
    #output6 = sess.run(accuracy)
#sess.close()

In [47]:
sess = tf.Session()

# 初期化
sess.run(tf.global_variables_initializer())

# 学習
for epoch in range(1000):
    sess.run(train_step, feed_dict={
        x:x_train,
        t:y_train
    })
    # 100回ごとに正解率を表示
    if epoch % 100 == 0:
        acc_val = sess.run(
            accuracy, feed_dict={
                x:x_train,
                t:y_train})
        print (f'epoch: {epoch}, Accuracy: {acc_val}')
        
        # h_thetaの確認
        mat = tf.matmul(x, W)
        y = tf.sigmoid(mat + b)
        
        mat = sess.run(mat, feed_dict={
            x:x_train,
            t:y_train
        })
        
        print(f'mat: {mat} mat.shape: {mat.shape}')

epoch: 0, Accuracy: 0.75
mat: [[0.]
 [0.]
 [0.]
 [0.]] mat.shape: (4, 1)
epoch: 100, Accuracy: 1.0
mat: [[0.       ]
 [1.7671355]
 [1.7671355]
 [3.534271 ]] mat.shape: (4, 1)
epoch: 200, Accuracy: 1.0
mat: [[0.       ]
 [2.7020476]
 [2.7020476]
 [5.404095 ]] mat.shape: (4, 1)
epoch: 300, Accuracy: 1.0
mat: [[0.       ]
 [3.3457708]
 [3.3457708]
 [6.6915417]] mat.shape: (4, 1)
epoch: 400, Accuracy: 1.0
mat: [[0.      ]
 [3.841228]
 [3.841228]
 [7.682456]] mat.shape: (4, 1)
epoch: 500, Accuracy: 1.0
mat: [[0.      ]
 [4.244355]
 [4.244355]
 [8.48871 ]] mat.shape: (4, 1)
epoch: 600, Accuracy: 1.0
mat: [[0.       ]
 [4.5839767]
 [4.5839767]
 [9.1679535]] mat.shape: (4, 1)
epoch: 700, Accuracy: 1.0
mat: [[0.       ]
 [4.8771544]
 [4.8771544]
 [9.754309 ]] mat.shape: (4, 1)
epoch: 800, Accuracy: 1.0
mat: [[ 0.      ]
 [ 5.134886]
 [ 5.134886]
 [10.269772]] mat.shape: (4, 1)
epoch: 900, Accuracy: 1.0
mat: [[ 0.       ]
 [ 5.3646903]
 [ 5.3646903]
 [10.729381 ]] mat.shape: (4, 1)


In [43]:
#学習結果が正しいか確認
classified = sess.run(correct_prediction, feed_dict={
    x:x_train,
    t:y_train
})

#出力yの確認
prob = sess.run(y, feed_dict={
    x:x_train,
    t:y_train
})

print(classified)

print(prob)

sess.close()

[[ True]
 [ True]
 [ True]
 [ True]]
[[1.9651403e-04]
 [4.9049813e-02]
 [4.9049813e-02]
 [9.3120378e-01]]


## 【問題1】スクラッチを振り返る  
どのような実装をしたかを列挙する
- 重みを初期化する必要があった
- エポックのループが必要だった
